In [1]:
import pandas

import numpy
import pygsheets
import datetime

In [2]:
#pandas.io.formats.excel.ExcelFormatter.header_style = None
excel_writer = pandas.ExcelWriter('EuropeGasTracker-Terminals-SummarySheets-'+str(datetime.date.today())+'.xlsx', engine='xlsxwriter')

import Terminals_Current dataset

In [6]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
#gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1BHiidrJnzh96Em3KrqBpCoWaJ3GB5OeCY9dHVaEDEFs') # Europe gas report version
#spreadsheet = gc.open_by_key('1tcS6Wd-Wp-LTDpLzFgJY_RSNDnbyubW3J_9HKIAys4A') # current version

#spreadsheet[1] "Gas Pipelines" tab is the second index
terms_df_orig = spreadsheet.worksheet('title', 'Terminals').get_as_df()

#get other relevant sheets
owners_df_orig = spreadsheet.worksheet('title', 'Owner').get_as_df()
owners_df_orig.set_index('ComboID', inplace=True)

In [7]:
terms_df_orig.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('--', numpy.nan, inplace=True)
owners_df_orig.replace('', numpy.nan, inplace=True)

# drop any terminals without a country?

In [8]:
region_df_orig = spreadsheet[3].get_as_df(encoding='latin1')
region_df_eu = region_df_orig.copy()[region_df_orig['EuropeanUnion']=='Yes']
region_df_egt = region_df_orig.copy()[region_df_orig['EuroGasTracker']=='Yes']
region_df_europe = region_df_orig.copy()[region_df_orig['Region']=='Europe']
region_df_euro_uk = region_df_orig.copy()[(region_df_orig['Region']=='Europe') | 
                                          (region_df_orig['Country'].isin(['United Kingdom','Israel']))]

In [9]:
region_df_touse = region_df_euro_uk.copy()

### create EU-specific dataframes for region, country_ratios, owners_df

In [ ]:
owners_df_touse = owners_df_orig.loc[owners_df_orig['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

terms_df_touse = terms_df_orig.loc[terms_df_orig['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

### set up info to analyze Owners tab

In [ ]:
# get a set list of all owners
owner_column_list = ['Owner1', 
                     'Owner2', 
                     'Owner3', 
                     'Owner4', 
                     'Owner5', 
                     'Owner6', 
                     'Owner7', 
                     'Owner8', 
                     'Owner9', 
                     'Owner10']

percent_column_list = ['Owner1%', 
                     'Owner2%', 
                     'Owner3%', 
                     'Owner4%', 
                     'Owner5%', 
                     'Owner6%', 
                     'Owner7%', 
                     'Owner8%', 
                     'Owner9%', 
                     'Owner10%']

owner_id_list = ['ID1', 
                 'ID2', 
                 'ID3', 
                 'ID4', 
                 'ID5', 
                 'ID6', 
                 'ID7', 
                 'ID8', 
                 'ID9', 
                 'ID10']

In [ ]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(set(region_df_touse['Country'].tolist()))
region_list = sorted(set(region_df_touse['Region'].tolist()))

In [ ]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

### no. export trains by country/region, project status

### no. export trains by country/region, project status - INCLUDING RE-EXPORTING FACILITIES

In [ ]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Facility']=='Export') & (terms_df_touse['Type1']=='LNG')]
terms_df_reexport_subset = terms_df_touse.copy()[(terms_df_touse['Facility']=='Import') 
                                                    & (terms_df_touse['Type1']=='LNG')
                                                    & (terms_df_touse['ReExport']=='Yes')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)
ntrains_reexport_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_reexport_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===region-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby('Region').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_reexport_subset_status = terms_df_reexport_subset.copy()[terms_df_reexport_subset['Status']==status]
    ntrains_reexport_by_country[status] = terms_df_reexport_subset_status.groupby('Country').size()

print('===region-level calculations===')
for status in status_list:
    print(status)
    terms_df_reexport_subset_status = terms_df_reexport_subset.copy()[terms_df_reexport_subset['Status']==status]
    ntrains_reexport_by_region[status] = terms_df_reexport_subset_status.groupby('Region').size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)
ntrains_reexport_by_country = ntrains_reexport_by_country.fillna(0)
ntrains_reexport_by_region = ntrains_reexport_by_region.fillna(0)

# calculate totals
totals_row = ntrains_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_region = ntrains_by_region.append(totals_row)

totals_row = ntrains_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_country = ntrains_by_country.append(totals_row)

totals_row = ntrains_reexport_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_reexport_by_country = ntrains_reexport_by_country.append(totals_row)

totals_row = ntrains_reexport_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_reexport_by_region = ntrains_reexport_by_region.append(totals_row)

# do Prop+Const sum
ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)
ntrains_reexport_by_region['In Development (Proposed + Construction)'] = ntrains_reexport_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_reexport_by_country['In Development (Proposed + Construction)'] = ntrains_reexport_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country.applymap(int).applymap(str)
ntrains_by_region = ntrains_by_region.applymap(int).applymap(str)
ntrains_reexport_by_country = ntrains_reexport_by_country.applymap(int).applymap(str)
ntrains_reexport_by_region = ntrains_reexport_by_region.applymap(int).applymap(str)

# then replace all zeros with empty string
ntrains_reexport_by_country = ntrains_reexport_by_country.applymap('({})'.format)
ntrains_reexport_by_region = ntrains_reexport_by_region.applymap('({})'.format)
ntrains_reexport_by_country = ntrains_reexport_by_country.replace('(0)','')
ntrains_reexport_by_region = ntrains_reexport_by_region.replace('(0)','')

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]
ntrains_reexport_by_country = ntrains_reexport_by_country[excel_status_list]
ntrains_reexport_by_region = ntrains_reexport_by_region[excel_status_list]

ntrains_by_region.index.name = 'Region'
ntrains_by_country.index.name = 'Country'
ntrains_reexport_by_region.index.name = 'Region'
ntrains_reexport_by_country.index.name = 'Country'

ntrains_by_region += ntrains_reexport_by_region
ntrains_by_country += ntrains_reexport_by_country

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG export trains by region')#, index=False)
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG export trains by country')#, index=False)

### no. import trains by country/region, project status

In [ ]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Facility']=='Import') & (terms_df_touse['Type1']=='LNG')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby('Region').size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)

ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]

ntrains_by_region.index.name = 'Region'
ntrains_by_country.index.name = 'Country'

totals_row = ntrains_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_region = ntrains_by_region.append(totals_row)

totals_row = ntrains_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_country = ntrains_by_country.append(totals_row)

ntrains_by_country = ntrains_by_country.applymap(int).applymap(str)
ntrains_by_region = ntrains_by_region.applymap(int).applymap(str)

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG import trains by region')
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG import trains by country')

### capacity (mtpa) of import trains by country/region, project status

In [ ]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Facility']=='Import') & (terms_df_touse['Type1']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby('Region')['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.name = 'Region'
cap_by_country.index.name = 'Country'

totals_row = cap_by_region.sum(axis=0)
totals_row.name = 'Total'
cap_by_region = cap_by_region.append(totals_row)

totals_row = cap_by_country.sum(axis=0)
totals_row.name = 'Total'
cap_by_country = cap_by_country.append(totals_row)

cap_by_region.to_excel(excel_writer, 'LNG import capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG import capacity by country')

### capacity (mtpa) of export trains by country/region, project status

In [ ]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Facility']=='Export') & (terms_df_touse['Type1']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby('Region')['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.name = 'Region'
cap_by_country.index.name = 'Country'

totals_row = cap_by_region.sum(axis=0)
totals_row.name = 'Total'
cap_by_region = cap_by_region.append(totals_row)

totals_row = cap_by_country.sum(axis=0)
totals_row.name = 'Total'
cap_by_country = cap_by_country.append(totals_row)

cap_by_region.to_excel(excel_writer, 'LNG export capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG export capacity by country')

## owner analysis

In [ ]:
owner_list = []
for column in owner_column_list:
    owner_list+=list(owners_df_orig[column])
owner_list = list(set(owner_list))
# remove empty (no owner)
owner_list.remove(numpy.nan)
unique_owner_list = sorted(owner_list)

#pandas.Series(unique_owner_list).to_excel('AllFuels'+'UniqueOwnersList.xlsx')

### ntrains export by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Facility']=='Export']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_ntrains_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        ntrain_fractions = numpy.array(row_fractions)

        for owner_idx in range(n_owners):
            owners_ntrains_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_fractions[owner_idx]

owners_ntrains_by_status_df.index.name = 'Parent Company'
owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

totals_row = owners_ntrains_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)

#owners_ntrains_by_status_df.to_excel('EuroGas'+'LNGExportTrainsByOwnerAndStatus.xlsx')
owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG export trains by owner')

### ntrains import by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Facility']=='Import']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_ntrains_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        ntrain_fractions = numpy.array(row_fractions)

        for owner_idx in range(n_owners):
            owners_ntrains_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_fractions[owner_idx]

owners_ntrains_by_status_df.index.name = 'Parent Company'
owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list]

totals_row = owners_ntrains_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)

#owners_ntrains_by_status_df.to_excel('EuroGas'+'LNGImportTrainsByOwnerAndStatus.xlsx')
owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG import trains by owner')

### capacity import by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Facility']=='Import']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_capacity_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        train_capacity = row['CapacityInMtpa']
        ntrain_capacity_fractions = numpy.array(row_fractions)*train_capacity

        for owner_idx in range(n_owners):
            owners_capacity_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_capacity_fractions[owner_idx]

owners_capacity_by_status_df.index.name = 'Parent Company'
owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

totals_row = owners_capacity_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)

#owners_capacity_by_status_df.to_excel('EuroGas'+'LNGImportTrainCapacityByOwnerAndStatus.xlsx', na_rep='--')
owners_capacity_by_status_df.to_excel(excel_writer, 'LNG import capacity by owner')

### capacity export by parent company, project status

In [ ]:
owners_df_subset = owners_df_touse.copy()[owners_df_touse['Facility']=='Export']

##################################################

owner_list_subset = []
for column in owner_column_list:
    owner_list_subset+=list(owners_df_subset[column])
owner_list_subset = list(set(owner_list_subset))
# remove empty (no owner)
owner_list_subset.remove(numpy.nan)
unique_owner_list_subset = sorted(owner_list_subset)

##################################################
# convert percents to fractions
##################################################
owners_df_fractions = owners_df_subset.copy()

for col in percent_column_list:
    owners_df_fractions[col] = owners_df_fractions[col].apply(lambda x: numpy.nan if x in [numpy.nan] 
                                          else x[:-1]).astype(float)/100
#df['col'] = df['col'].apply(lambda x: np.nan if x in ['-'] else x[:-1]).astype(float)/100


##################################################
# create train count by owner, status
##################################################
no_owner_info_count = 0
owners_capacity_by_status_df = pandas.DataFrame(0.0, index=unique_owner_list_subset, columns=status_list)

for status in status_list:
    
    owners_df_temporary = owners_df_fractions.copy()[owners_df_fractions['Status']==status]
    
    for idx,row in owners_df_temporary.iterrows():
        
        ### how many owners are there?
        row_owners = list(row[owner_column_list])
        row_owners = [i for i in row_owners if str(i)!='nan']
        n_owners = row_owners.__len__()
        row_fractions = list(row[percent_column_list])
        row_fractions = [i for i in row_fractions if str(i)!='nan']

        ### if there are no owners listed, continue to next loop iteration
        if n_owners==0:
            no_owner_info_count+=1
            continue # doesn't complete the rest of the ifs

        ### now if row fractions is an empty list, create equal fractions list instead
        if row_fractions==[]:
            row_fractions = [1/n_owners]*n_owners

        train_capacity = row['CapacityInMtpa']
        ntrain_capacity_fractions = numpy.array(row_fractions)*train_capacity

        for owner_idx in range(n_owners):
            owners_capacity_by_status_df.loc[row_owners[owner_idx]][status]+=ntrain_capacity_fractions[owner_idx]

owners_capacity_by_status_df.index.name = 'Parent Company'
owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list]

totals_row = owners_capacity_by_status_df.sum(axis=0)
totals_row.name = 'Total'
owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)

#owners_capacity_by_status_df.to_excel('EuroGas'+'LNGExportTrainCapacityByOwnerAndStatus.xlsx', na_rep='--')
owners_capacity_by_status_df.to_excel(excel_writer, 'LNG export capacity by owner')

### no. of operating terminals by start year, facility type (1980–2021)

In [ ]:
terms_started_eu = terms_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

year_indices = list(range(1980,2022))

terms_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export trains'])

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Type1']=='LNG') & (terms_started_eu['Facility']=='Import')]
terms_by_start_year_df['Import terminals'] = terms_started_eu.groupby('StartYearEarliest').size()

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Type1']=='LNG') & (terms_started_eu['Facility']=='Export')]
terms_by_start_year_df['Export trains'] = terms_started_eu.groupby('StartYearEarliest').size()

terms_by_start_year_df.index.name = 'Start year'
terms_by_start_year_df.replace(numpy.nan,0,inplace=True)

totals_row = terms_by_start_year_df.sum(axis=0)
totals_row.name = 'Total'
terms_by_start_year_df = terms_by_start_year_df.append(totals_row)

terms_by_start_year_df.to_excel(excel_writer, 'LNG terminals by start year')

### capacity of operating terminals by start year, facility type (1980–2021)

In [ ]:
terms_started_eu = terms_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

year_indices = list(range(1980,2022))

capacity_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export trains'])

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Type1']=='LNG') & (terms_started_eu['Facility']=='Import')]
capacity_by_start_year_df['Import terminals'] = terms_started_eu.groupby('StartYearEarliest')['CapacityInMtpa'].sum()

terms_started_eu = terms_started_eu[(terms_started_eu['Status'].isin(['Operating'])) &
                              (terms_started_eu['Type1']=='LNG') & (terms_started_eu['Facility']=='Export')]
capacity_by_start_year_df['Export trains'] = terms_started_eu.groupby('StartYearEarliest')['CapacityInMtpa'].sum()

capacity_by_start_year_df.index.name = 'Start year'
capacity_by_start_year_df.replace(numpy.nan,0,inplace=True)

totals_row = capacity_by_start_year_df.sum(axis=0)
totals_row.name = 'Total'
capacity_by_start_year_df = capacity_by_start_year_df.append(totals_row)

capacity_by_start_year_df.to_excel(excel_writer, 'LNG capacity by start year')

## save excel file

In [ ]:
excel_writer.save()